<a href="https://colab.research.google.com/github/gzholtkevych/URM-Programming/blob/main/URM_en.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<H1><b>Unlimited Register Machine</b></H1>

**U**nlimited **R**egister **M**achine (URM) is a realization of the approach to programming based on the automaton concept.
URM is an easier-to-understand alternative to a Turing machine.

It has (to a certain extent) the same capabilities as the Turing machine, to which URM is logically equivalent.
The URM was presented in an article published in 1963

>[John C. Shepherdson and H.E. Sturgis: *Computability of Recursive Functions*. J. ACM Vol. 10, no. 2, pp. 217–255.](https://dl.acm.org/doi/pdf/10.1145/321160.321170)

# Preparing a notebook to use

In [1]:
from google.colab import drive
drive.mount('/content/drive')

import sys
sys.path.append('/content/drive/MyDrive/ColabNotebooks/Computability')

ModuleNotFoundError: No module named 'google.colab'

In [6]:
from typing import Dict, Any, Optional, Tuple, List
from typing_extensions import Self
from functools import reduce
from compy.nat import nat

# Memory of UMR

## Registers

URM has an enumerated set of ***registers***, each of which can store a natural number $n\in\mathbb{N}$, where $\mathbb{N}=\{0,1,2,\dotsc\} $.

Each URM program can use only a finite set of registers.

Registers are usually denoted by a capital letter $R$ with an index: $R_{0},R_{1},R_{2},\dotsc$.
The index (which is a natural number) is called the ***register index***.
The number holding register $R_{n}$ is usually denoted by $r_{n}$.

Registers are unlimited in the following two senses:

- even though a URM program can use only a finite set of registers, there is no upper limit for the number of registers used by the programs;
- there is no upper limit for numbers stored by registers.

## Memory states

The state of the memory is determined by the values ​​stored in the registers.

## Software implementation

The memory of URM is implemented as class `memory` inherited from the built-in type `list`.

The property `size` equals the length of the corresponding list.

The methods `read()` and `write()` provide correct reading from and writing to the memory.

In [7]:
class memory(list):

    def __new__(cls) -> Self:
        return super().__new__(cls, [])

    @property
    def size(self) -> int:
        return len(self)

    def read(self, addr: int) -> nat:
        try:
            n = nat(addr)
        except ValueError:
            raise ValueError("memory.read() error! Bad argument")
        try:
            return self[n]
        except IndexError:
            return nat(0)

    def write(self, addr: int, value: Any) -> None:
        try:
            n, v = nat(addr), nat(value)
        except ValueError:
            raise ValueError("memory.write() error! Bad argument(s)")
        diff = self.size - n
        if diff <= 0:  # access to a register that is not yet allocated
            self += (1 - diff) * [nat(0)]  # allocate additional registers
        self[n] = v


## URM-programs

***Execution states***, that is, the value of the registers and the instruction counter, is changed by programs.

A ***URM program*** is a finite list of ***instructions***.

Program instructions are written in a certain order and numbered with positive natural numbers $1,2,3,\dotsc$.<br/>
**Please note!** Instruction number 0 does not exist in the program.

The instruction number for historical reasons is called the ***program line number***.

## Instruction counter

The ***instruction counter*** is an additional register that stores a natural number.
This register is denoted by $IC$ and its contents are $ic$.

<table width="800pt">
<tr><th width="100pt">Name</th>
<th width=100pt>Mnemonic code</th>
<th width="200pt">Effect</th>
<th>Description</th></tr>
<tr><td>Zero</td>
<td align="center">$\mathtt{Z}(n)$</td>
<td align="center">$R_n\gets 0$<br/>$IC\gets ic+1$</td>
<td>Replace the value in $R_{n}$ with $0$ and jump to the nextline of the<br/>program</td></tr>
<tr><td>Successor</td>
<td align="center">$\mathtt{S}(n)$</td>
<td align="center">$R_n\gets r_n+1$<br/>$IC\gets ic+1$</td>
<td>Increase the value of register $R_{n}$ by $1$ and jump to the next line of<br/>the program</td></tr>
<tr><td>Copy</td>
<td align="center">$\mathtt{C}(m,n)$</td>
<td align="center">$R_n\gets r_m$<br/>$IC\gets ic+1$</td>
<td>Replace the value in $R_{n}$ with the content of $R_{m}$ (keeping the<br/>content of $R_{m}$) and jump to the next line of the program</td></tr>
<tr><td>Jump</td>
<td align="center">$\mathtt{J}(m,n,k)$</td>
<td align="center">$IC\gets k\ \mathtt{if}\ r_m=r_n\ \mathtt{else}\ ic+1$</td>
<td>If the values ​​stored in $R_{m}$ and $R_{n}$ match, then jump to the<br/>program line with number $k$, otherwise to the next program line</td></tr>
</table>

## Software implementation

The `ins` class provides the data type implementation inhabited by the URM instructions.

- The instruction `Z(n)` is represented by a tuple `(0, n)`.
- The instruction `S(n)` is represented by a tuple `(1, n)`.
- The instruction `C(m, n)` is represented by the tuple `(2, m, n)`.
- The instruction `J(m, n, k)` is represented by the tuple `(3, m, n, k)`.

To correctly represent instructions by strings, the `__str__()` method is overloaded for this class.

The class methods `Z()`, `S()`, `C()` and `J()` provide convenient instruction creation.

In [8]:
class ins(tuple):

    def __new__(cls, *args: Tuple[int]) -> Self:
        if not (2 <= len(args) <= 4):
            raise ValueError("ins() error! Invalid number of arguments")
        if not all(type(arg) == int for arg in args):
            raise ValueError("ins() error! Invalid type of argument(s)")
        if args[0] > 3:
            raise ValueError("ins() error! Invalid instruction code")
        if not all(arg >= 0 for arg in args[1 :]):
            raise ValueError("ins() error! Invalid value of operand(s)")
        return super().__new__(cls, args)

    def __str__(self) -> str:
        if self[0] == 0:
            return f"Z({self[1]})"
        if self[0] == 1:
            return f"S({self[1]})"
        if self[0] == 2:
            return f"C{self[1 :]}"
        # self[0] == 3
        return f"J{self[1 :]}"

    @classmethod
    def Z(cls, n: Any) -> Self:
        try:
            return ins(0, n)
        except ValueError:
            raise ValueError("Z() error! Bad operand")

    @classmethod
    def S(cls, n: Any) -> Self:
        try:
            return ins(1, n)
        except ValueError:
            raise ValueError("S() error! Bad operand")

    @classmethod
    def C(cls, m: Any, n: Any) -> Self:
        try:
            return ins(2, m, n)
        except ValueError:
            raise ValueError("C() error! Bad operand(s)")

    @classmethod
    def J(cls, m: Any, n: Any, k: Any) -> Self:
        try:
            return ins(3, m, n, k)
        except ValueError:
            raise ValueError("J() error! Bad operand(s)")


## Instructions execution

The execution of `instruction: ins` if the memory state is `memory_state` implements by calling `apply(instruction, to=memory_state)`, which provides returning the pair `(how_to_change_IC, new_memory_state)`.

The value of `how_to_change_IC` is equal to

- `None`, if after the instruction `instruction` it is necessary to execute the instruction in the next line of the program;
- the line number of the instruction that will be executed next, if it is not the next in order in the program.

The value `new_memory_state` is the state of the memory after the execution of the instruction `instruction`.

In [9]:
def apply(i: ins, to: memory) -> Tuple[Optional[int], memory]:
    if type(to) != memory:
        raise ValueError("do() error! Invalid type of memory")
    if type(i) != ins:
        raise ValueError("do() error! Invalid type of instruction")
    if i[0] == 0:
        to.write(i[1], 0)
        return (None, to)
    if i[0] == 1:
        to.write(i[1], to.read(i[1]) + 1)
        return (None, to)
    if i[0] == 2:
        to.write(i[2], to.read(i[1]))
        return (None, to)
    # i[0] == 3
    return (i[3] if to.read(i[1]) == to.read(i[2]) else None, to)

In [10]:
m = memory()
for n in range(10):
    m.write(n, n)
print(f"{13 * ' '}m = {m}")
next, m = apply(ins.Z(9), to=m)
print(f"next = {next}; m = {m}")
next, m = apply(ins.S(9), to=m)
print(f"next = {next}; m = {m}")
next, m = apply(ins.C(9, 0), to=m)
print(f"next = {next}; m = {m}")
next, m = apply(ins.J(1, 9, 5), to=m)
print(f"next = {next};    m = {m}")
next, m = apply(ins.J(2, 3, 10), to=m)
print(f"next = {next}; m = {m}")

             m = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
next = None; m = [0, 1, 2, 3, 4, 5, 6, 7, 8, 0]
next = None; m = [0, 1, 2, 3, 4, 5, 6, 7, 8, 1]
next = None; m = [1, 1, 2, 3, 4, 5, 6, 7, 8, 1]
next = 5;    m = [1, 1, 2, 3, 4, 5, 6, 7, 8, 1]
next = None; m = [1, 1, 2, 3, 4, 5, 6, 7, 8, 1]


# URM-programs and their execution

## Translation of a source text into an instruction list

The function `compile()` converts the text into a list of URM instructions.

In [11]:
def compile(text: str) -> Tuple[ins]:
    if not isinstance(text, str):
        raise ValueError("compile() error! Invalid argument type")
    # split `text` into the list of non empty lines
    lines = [line for line in [item.strip() for
                               item in text.split('\n')] if
             line]
    # the function compiles a line
    def compile_line(line: str) -> ins:
        # remove the tail of 'line' beginning with ')'
        item, sep, _ = line.partition(')')
        if not sep:  # ')' is absent in the line
            raise ValueError("compile_line() error! Bad instruction format")
        code, sep, item = item.partition('(')
        if not sep:  # '(' is absent in the line
            raise ValueError("compile_line() error! Bad instruction format")
        if code.strip() == 'Z':
            try:  # to recognize Z-instruction
                return ins.Z(int(item))
            except ValueError:
                raise ValueError("compile_line() error! Bad Z-instruction")
        elif code.strip() == 'S':
            try:  # to recognize S-instruction
                return ins.S(int(item))
            except ValueError:
                raise ValueError("compile_line() error! Bad S-instruction")
        elif code.strip() == 'C':
            op1, sep, item = item.partition(',')
            try:  # to recognize C-instruction
                return ins.C(int(op1), int(item))
            except ValueError:
                raise ValueError("compile_line() error! Bad C-instruction")
        elif code.strip() == 'J':
            op1, sep, item = item.partition(',')
            op2, sep, item = item.partition(',')
            try:  # to recognize J-instruction
                return ins.J(int(op1), int(op2), int(item))
            except:
                raise ValueError("compile_line() error! Bad J-instruction")
        raise ValueError("compile_line() error! Bad instruction format")
    # end of compile_line()
    return tuple(compile_line(line) for line in lines)

An example of using the `compile()` function.

In [12]:
text = """
    C(2, 0)
    Z (2)

    J(1, 2, 0)  check loop condition
        S(0)
        S(2)
    J(0, 0, 3)  repeat loop
"""

ins_tuple = compile(text)
for ni, i in enumerate(ins_tuple):
    print(f"{ni + 1:3d}: {i}")

  1: C(2, 0)
  2: Z(2)
  3: J(1, 2, 0)
  4: S(0)
  5: S(2)
  6: J(0, 0, 3)


## URM-programs

### Software implementation of a URM-program

In [13]:
class program(tuple):

    def __new__(cls, instructions: Tuple[ins]) -> Self:
        if not type(instructions) == tuple:
            raise ValueError("program() error! Invalid argument type")
        if not all(type(instruction) == ins for instruction in instructions):
            raise ValueError("program() error! Invalid type of argument member")
        return super().__new__(cls, instructions)

    def __str__(self):
        return "\n".join([f"{ni + 1: 3d}: {i}" for ni, i in enumerate(self)])

    @property
    def length(self):
        return len(self)

    @property
    def haddr(self):
        temp = 0
        for i in self:
            temp = max(temp, *i[1 : 3])
        return temp

    def get_instruction(self, lineno: int) -> Optional[ins]:
        if type(lineno) != int:
            raise ValueError("program.get_instruction() error! Bad line number")
        if lineno < 1:
            return None
        try:
            return self[lineno - 1]
        except IndexError:
            return None

In [14]:
p = program(compile(text))
print(f"Program text:\n{p}")
print(f"Program length = {p.length}")
print(f"Program highest address = {p.haddr}")

Program text:
  1: C(2, 0)
  2: Z(2)
  3: J(1, 2, 0)
  4: S(0)
  5: S(2)
  6: J(0, 0, 3)
Program length = 6
Program highest address = 2


### Execution of a URM-program

In [116]:
def run(prgm: program, *data: Tuple[Any]) -> nat:
    if type(prgm) != program:
        raise ValueError("run() error! Bad program")
    try:
        data = (nat(x) for x in data)
    except:
        raise ValueError("run() error! Invalid data")
    m = memory()
    for addr, value in enumerate(data):
        m.write(addr + 1, value)
    ic = 1
    while True:
        print(m)
        i = prgm.get_instruction(ic)
        if i is None:
            return m.read(0)
        q, m = apply(i, to=m)
        ic = ic + 1 if q is None else q

In [16]:
report = [(x, y, run(p, x, y)) for x in range(10) for y in range(10)]
report_items = [" ".join(
    (f"{x}", "+", f"{y}", "=", f"{z:2d}")) for (x, y, z) in report]
report_lines = [" : ".join(report_items[ic + c * 20] for c in range(5)) for
                ic in range(20)]
for line in report_lines:
    print(line)

[0, 0, 0]
[0, 0, 0]
[0, 0, 0]
[0, 0, 0]
[0, 0, 1]
[1, 0, 1]
[1, 0, 0]
[1, 0, 0]
[0, 0, 2]
[2, 0, 2]
[2, 0, 0]
[2, 0, 0]
[0, 0, 3]
[3, 0, 3]
[3, 0, 0]
[3, 0, 0]
[0, 0, 4]
[4, 0, 4]
[4, 0, 0]
[4, 0, 0]
[0, 0, 5]
[5, 0, 5]
[5, 0, 0]
[5, 0, 0]
[0, 0, 6]
[6, 0, 6]
[6, 0, 0]
[6, 0, 0]
[0, 0, 7]
[7, 0, 7]
[7, 0, 0]
[7, 0, 0]
[0, 0, 8]
[8, 0, 8]
[8, 0, 0]
[8, 0, 0]
[0, 0, 9]
[9, 0, 9]
[9, 0, 0]
[9, 0, 0]
[0, 1, 0]
[0, 1, 0]
[0, 1, 0]
[0, 1, 0]
[1, 1, 0]
[1, 1, 1]
[1, 1, 1]
[1, 1, 1]
[0, 1, 1]
[1, 1, 1]
[1, 1, 0]
[1, 1, 0]
[2, 1, 0]
[2, 1, 1]
[2, 1, 1]
[2, 1, 1]
[0, 1, 2]
[2, 1, 2]
[2, 1, 0]
[2, 1, 0]
[3, 1, 0]
[3, 1, 1]
[3, 1, 1]
[3, 1, 1]
[0, 1, 3]
[3, 1, 3]
[3, 1, 0]
[3, 1, 0]
[4, 1, 0]
[4, 1, 1]
[4, 1, 1]
[4, 1, 1]
[0, 1, 4]
[4, 1, 4]
[4, 1, 0]
[4, 1, 0]
[5, 1, 0]
[5, 1, 1]
[5, 1, 1]
[5, 1, 1]
[0, 1, 5]
[5, 1, 5]
[5, 1, 0]
[5, 1, 0]
[6, 1, 0]
[6, 1, 1]
[6, 1, 1]
[6, 1, 1]
[0, 1, 6]
[6, 1, 6]
[6, 1, 0]
[6, 1, 0]
[7, 1, 0]
[7, 1, 1]
[7, 1, 1]
[7, 1, 1]
[0, 1, 7]
[7, 1, 7]
[7, 1, 0]
[7, 1, 0]


[11, 7, 4]
[11, 7, 5]
[11, 7, 5]
[11, 7, 5]
[12, 7, 5]
[12, 7, 6]
[12, 7, 6]
[12, 7, 6]
[13, 7, 6]
[13, 7, 7]
[13, 7, 7]
[13, 7, 7]
[0, 7, 7]
[7, 7, 7]
[7, 7, 0]
[7, 7, 0]
[8, 7, 0]
[8, 7, 1]
[8, 7, 1]
[8, 7, 1]
[9, 7, 1]
[9, 7, 2]
[9, 7, 2]
[9, 7, 2]
[10, 7, 2]
[10, 7, 3]
[10, 7, 3]
[10, 7, 3]
[11, 7, 3]
[11, 7, 4]
[11, 7, 4]
[11, 7, 4]
[12, 7, 4]
[12, 7, 5]
[12, 7, 5]
[12, 7, 5]
[13, 7, 5]
[13, 7, 6]
[13, 7, 6]
[13, 7, 6]
[14, 7, 6]
[14, 7, 7]
[14, 7, 7]
[14, 7, 7]
[0, 7, 8]
[8, 7, 8]
[8, 7, 0]
[8, 7, 0]
[9, 7, 0]
[9, 7, 1]
[9, 7, 1]
[9, 7, 1]
[10, 7, 1]
[10, 7, 2]
[10, 7, 2]
[10, 7, 2]
[11, 7, 2]
[11, 7, 3]
[11, 7, 3]
[11, 7, 3]
[12, 7, 3]
[12, 7, 4]
[12, 7, 4]
[12, 7, 4]
[13, 7, 4]
[13, 7, 5]
[13, 7, 5]
[13, 7, 5]
[14, 7, 5]
[14, 7, 6]
[14, 7, 6]
[14, 7, 6]
[15, 7, 6]
[15, 7, 7]
[15, 7, 7]
[15, 7, 7]
[0, 7, 9]
[9, 7, 9]
[9, 7, 0]
[9, 7, 0]
[10, 7, 0]
[10, 7, 1]
[10, 7, 1]
[10, 7, 1]
[11, 7, 1]
[11, 7, 2]
[11, 7, 2]
[11, 7, 2]
[12, 7, 2]
[12, 7, 3]
[12, 7, 3]
[12, 7, 3]
[13, 7, 3]
[

# Manipulating URM-programs

## Normalization of URM-programs

In [21]:
def normalize(P: program) -> program:
    """
    The function normalizes a URM program by ensuring all jump instructions
    have valid targets. If a jump target k is not in range [1, program_size],
    it is set to program_size + 1.

    Arguments
        P: program

    Returns
        program
    """
    if type(P) != program:
        raise ValueError("normalize() error! Invalid argument type")
    if P.length == 0:
        return P
    normalized_instructions = []
    program_size = P.length
    for i, instruction in enumerate(P):
        if instruction[0] == 3:
            m, n, k = instruction[1:]
            if not (1 <= k <= program_size):
                k = program_size + 1
            normalized_instructions.append(ins.J(m, n, k))
        else:
            normalized_instructions.append(instruction)

    return program(tuple(normalized_instructions))

In [22]:
# Check normalized program
p = program(compile(text))
print(f"Original program:\n{p}")
p_n = normalize(p)
report = [(x, y, run(p_n, x, y)) for x in range(10) for y in range(10)]
for x, y, ret in report:
    assert (x + y) == int(ret), f"Failed: {x} + {y} = {ret} (expected {x+y})"
print(f"Normalized program:\n{p_n}")

Original program:
  1: C(2, 0)
  2: Z(2)
  3: J(1, 2, 0)
  4: S(0)
  5: S(2)
  6: J(0, 0, 3)
[0, 0, 0]
[0, 0, 0]
[0, 0, 0]
[0, 0, 0]
[0, 0, 1]
[1, 0, 1]
[1, 0, 0]
[1, 0, 0]
[0, 0, 2]
[2, 0, 2]
[2, 0, 0]
[2, 0, 0]
[0, 0, 3]
[3, 0, 3]
[3, 0, 0]
[3, 0, 0]
[0, 0, 4]
[4, 0, 4]
[4, 0, 0]
[4, 0, 0]
[0, 0, 5]
[5, 0, 5]
[5, 0, 0]
[5, 0, 0]
[0, 0, 6]
[6, 0, 6]
[6, 0, 0]
[6, 0, 0]
[0, 0, 7]
[7, 0, 7]
[7, 0, 0]
[7, 0, 0]
[0, 0, 8]
[8, 0, 8]
[8, 0, 0]
[8, 0, 0]
[0, 0, 9]
[9, 0, 9]
[9, 0, 0]
[9, 0, 0]
[0, 1, 0]
[0, 1, 0]
[0, 1, 0]
[0, 1, 0]
[1, 1, 0]
[1, 1, 1]
[1, 1, 1]
[1, 1, 1]
[0, 1, 1]
[1, 1, 1]
[1, 1, 0]
[1, 1, 0]
[2, 1, 0]
[2, 1, 1]
[2, 1, 1]
[2, 1, 1]
[0, 1, 2]
[2, 1, 2]
[2, 1, 0]
[2, 1, 0]
[3, 1, 0]
[3, 1, 1]
[3, 1, 1]
[3, 1, 1]
[0, 1, 3]
[3, 1, 3]
[3, 1, 0]
[3, 1, 0]
[4, 1, 0]
[4, 1, 1]
[4, 1, 1]
[4, 1, 1]
[0, 1, 4]
[4, 1, 4]
[4, 1, 0]
[4, 1, 0]
[5, 1, 0]
[5, 1, 1]
[5, 1, 1]
[5, 1, 1]
[0, 1, 5]
[5, 1, 5]
[5, 1, 0]
[5, 1, 0]
[6, 1, 0]
[6, 1, 1]
[6, 1, 1]
[6, 1, 1]
[0, 1, 6]
[6, 1, 6]
[6, 1, 

[8, 9, 7]
[8, 9, 7]
[8, 9, 7]
[9, 9, 7]
[9, 9, 8]
[9, 9, 8]
[9, 9, 8]
[10, 9, 8]
[10, 9, 9]
[10, 9, 9]
[10, 9, 9]
[0, 9, 2]
[2, 9, 2]
[2, 9, 0]
[2, 9, 0]
[3, 9, 0]
[3, 9, 1]
[3, 9, 1]
[3, 9, 1]
[4, 9, 1]
[4, 9, 2]
[4, 9, 2]
[4, 9, 2]
[5, 9, 2]
[5, 9, 3]
[5, 9, 3]
[5, 9, 3]
[6, 9, 3]
[6, 9, 4]
[6, 9, 4]
[6, 9, 4]
[7, 9, 4]
[7, 9, 5]
[7, 9, 5]
[7, 9, 5]
[8, 9, 5]
[8, 9, 6]
[8, 9, 6]
[8, 9, 6]
[9, 9, 6]
[9, 9, 7]
[9, 9, 7]
[9, 9, 7]
[10, 9, 7]
[10, 9, 8]
[10, 9, 8]
[10, 9, 8]
[11, 9, 8]
[11, 9, 9]
[11, 9, 9]
[11, 9, 9]
[0, 9, 3]
[3, 9, 3]
[3, 9, 0]
[3, 9, 0]
[4, 9, 0]
[4, 9, 1]
[4, 9, 1]
[4, 9, 1]
[5, 9, 1]
[5, 9, 2]
[5, 9, 2]
[5, 9, 2]
[6, 9, 2]
[6, 9, 3]
[6, 9, 3]
[6, 9, 3]
[7, 9, 3]
[7, 9, 4]
[7, 9, 4]
[7, 9, 4]
[8, 9, 4]
[8, 9, 5]
[8, 9, 5]
[8, 9, 5]
[9, 9, 5]
[9, 9, 6]
[9, 9, 6]
[9, 9, 6]
[10, 9, 6]
[10, 9, 7]
[10, 9, 7]
[10, 9, 7]
[11, 9, 7]
[11, 9, 8]
[11, 9, 8]
[11, 9, 8]
[12, 9, 8]
[12, 9, 9]
[12, 9, 9]
[12, 9, 9]
[0, 9, 4]
[4, 9, 4]
[4, 9, 0]
[4, 9, 0]
[5, 9, 0]
[5, 9, 1]
[5, 9,

## Concatenation of URM-programs

In [23]:
def concat(P: program, Q: program) -> program:
    """
    The function returns the program that operates as P until
    its termination and then as Q.

    Arguments
        P, Q: program

    Returns
        program
    """
    if not (isinstance(P, program) and isinstance(Q, program)):
        raise ValueError("concat() error! Invalid program type")
    if P.length == 0:
        return Q

    P_normalized = normalize(P)
    offset = P_normalized.length
    Q_adjusted = []
    for instruction in Q:
        if instruction[0] == 3:  # Jump instruction
            m, n, k = instruction[1:]
            new_k = k + offset if k > 0 else k
            Q_adjusted.append(ins.J(m, n, new_k))
        else:
            Q_adjusted.append(instruction)

    cat = list(P_normalized) + Q_adjusted

    return program(tuple(cat))

In [24]:
text_P = """Z(0)
            C(1,2)
            S(0)
            J(3,4,0)
        """
text_Q = """
            S(0)
            J(3,4,0)
            Z(0)
            C(1,0)
        """
P, Q = program(compile(text_P)), program(compile(text_Q))
C = concat(P, Q)
print(f'Concatenation: \n{C}')

Concatenation: 
  1: Z(0)
  2: C(1, 2)
  3: S(0)
  4: J(3, 4, 5)
  5: S(0)
  6: J(3, 4, 0)
  7: Z(0)
  8: C(1, 0)


In [25]:
# Pretty print
COLUMN_WIDTH = 13
SEPARATOR = '\t'

print("Programs Concatenation Comparison:")
print("-" * 60)

print(f"{'Program P':13}{SEPARATOR}{'Program Q':13}{SEPARATOR}"
      f"{'P concatenated with Q'}")
print("-" * 60)

for line in range(P.length):
    print(f"{str(P[line]):{COLUMN_WIDTH}}"
          f"{SEPARATOR}{COLUMN_WIDTH * ' '}"
          f"{SEPARATOR}{C[line]}")

for line in range(Q.length):
    print(f"{COLUMN_WIDTH * ' '}"
          f"{SEPARATOR}{str(Q[line]):{COLUMN_WIDTH}}"
          f"{SEPARATOR}{C[P.length + line]}")

print("-" * 60)

Programs Concatenation Comparison:
------------------------------------------------------------
Program P    	Program Q    	P concatenated with Q
------------------------------------------------------------
Z(0)         	             	Z(0)
C(1, 2)      	             	C(1, 2)
S(0)         	             	S(0)
J(3, 4, 0)   	             	J(3, 4, 5)
             	S(0)         	S(0)
             	J(3, 4, 0)   	J(3, 4, 0)
             	Z(0)         	Z(0)
             	C(1, 0)      	C(1, 0)
------------------------------------------------------------


## Moving URM-program in Memory

In [26]:
def move(P: program, off: int=0) -> program:
    """
    The function returns the program that operates as P in the registers
    R[off], R[off + 1], ...

    Arguments
        P: program
        off: int {off >= 0}
    Returns
        program
    """
    if not isinstance(P, program):
        raise ValueError("move() error! Invalid program type")
    if not isinstance(off, int) or off < 0:
        raise ValueError("move() error! Invalid offset")
    # end of checking arguments
    if P.length == 0 or off == 0:
        return P
    moved_instructions = []
    for instruction in P:
        # Handle different op
        if instruction[0] == 0:  # Z()
            moved_instructions.append(ins.Z(instruction[1] + off))
        elif instruction[0] == 1:  # S()
            moved_instructions.append(ins.S(instruction[1] + off))
        elif instruction[0] == 2:  # C()
            moved_instructions.append(ins.C(instruction[1] + off,
                                            instruction[2] + off))
        else:  # J()
            moved_instructions.append(ins.J(instruction[1] + off,
                                            instruction[2] + off,
                                            instruction[3]))
    return program(tuple(moved_instructions))

In [27]:
# Make the move of an add() program to memory offset by 5
text_add = """
    C(2, 0)
    Z (2)

    J(1, 2, 0)  check loop condition
        S(0)
        S(2)
    J(0, 0, 3)  repeat loop
"""
add = program(compile(text_add))
offset = 5
add_moved = move(add, offset)

# Pretty print
COLUMN_WIDTH = 13
SEPARATOR = '\t'

print("Original vs Moved Program (offset=5):")
print("-" * 60)
print(f"{'Original':13}{SEPARATOR}{'Moved':13}{SEPARATOR}")
print("-" * 60)

for line in range(add.length):
   print(f"{str(add[line]):{COLUMN_WIDTH}}"
         f"{SEPARATOR}{str(add_moved[line]):{COLUMN_WIDTH}}")

print("-" * 60)

Original vs Moved Program (offset=5):
------------------------------------------------------------
Original     	Moved        	
------------------------------------------------------------
C(2, 0)      	C(7, 5)      
Z(2)         	Z(7)         
J(1, 2, 0)   	J(6, 7, 0)   
S(0)         	S(5)         
S(2)         	S(7)         
J(0, 0, 3)   	J(5, 5, 3)   
------------------------------------------------------------


In [28]:
# Check moved program
args = (2, 6)
args_padding = tuple([0] * offset) + args
# Add a C() to tail for copying result
cp = program(tuple([ins.C(offset, 0)]))
add_moved_c = concat(add_moved, cp)
print(f"{add_moved_c}")
ret = run(add_moved_c, *args_padding)
# Check
assert args[0] + args[1] == int(ret), "Error, check the move function"
print(f"\n{args[0]} + {args[1]} = {ret}")

  1: C(7, 5)
  2: Z(7)
  3: J(6, 7, 7)
  4: S(5)
  5: S(7)
  6: J(5, 5, 3)
  7: C(5, 0)
[0, 0, 0, 0, 0, 0, 2, 6]
[0, 0, 0, 0, 0, 6, 2, 6]
[0, 0, 0, 0, 0, 6, 2, 0]
[0, 0, 0, 0, 0, 6, 2, 0]
[0, 0, 0, 0, 0, 7, 2, 0]
[0, 0, 0, 0, 0, 7, 2, 1]
[0, 0, 0, 0, 0, 7, 2, 1]
[0, 0, 0, 0, 0, 7, 2, 1]
[0, 0, 0, 0, 0, 8, 2, 1]
[0, 0, 0, 0, 0, 8, 2, 2]
[0, 0, 0, 0, 0, 8, 2, 2]
[0, 0, 0, 0, 0, 8, 2, 2]
[8, 0, 0, 0, 0, 8, 2, 2]

2 + 6 = 8


## Alternating Execution of Programs

In [324]:
def alter(P: program, Q: program) -> program:
    """
    The function returns the program implementing the following algorithm:
        1. Provide to execute P and Q in disjoint memory segments.
        2. Do one step of P.
        3. If the execution of P terminates, copy its result to R0.
        4. Do one step of Q.
        5. If the execution of Q terminates, copy its result to R1.
        6. Repeat from step 2.
    Arguments:
        P: program
        Q: program
    Returns:
        program
    """
    if not isinstance(P, program) or not isinstance(Q, program):
        raise ValueError("alter() error! Invalid program type")

    # Step 1: Move Q to a disjoint memory segment
    P_haddr = P.haddr
    offset_Q = P_haddr + 1  # Offset for Q's registers
    P_moved = move(P, 0)    # P remains at its original position
    Q_moved = move(Q, offset_Q)

    # Define result registers
    r_p = 0                # P's result register (after move, remains 0)
    r_q = offset_Q         # Q's result register (after move)

    # Step 2: Interleave instructions and build line maps
    interleaved_instructions = []
    line_map_P = {}
    line_map_Q = {}
    instr_origin = []  # Keep track of the origin ('P' or 'Q') of each instruction

    idx = 1
    P_idx = 1
    Q_idx = 1
    P_length = P_moved.length
    Q_length = Q_moved.length
    while P_idx <= P_length or Q_idx <= Q_length:
        if P_idx <= P_length:
            line_map_P[P_idx] = idx
            interleaved_instructions.append(P_moved[P_idx - 1])
            instr_origin.append('P')
            idx += 1
            P_idx += 1
        if Q_idx <= Q_length:
            line_map_Q[Q_idx] = idx
            interleaved_instructions.append(Q_moved[Q_idx - 1])
            instr_origin.append('Q')
            idx += 1
            Q_idx += 1

    # Step 3: Add termination code for P and Q
    t_p = idx
    interleaved_instructions.append(ins.C(r_p, 0))  # Copy P's result to R0
    instr_origin.append('Termination_P')
    idx += 1

    t_q = idx
    interleaved_instructions.append(ins.C(r_q, 1))  # Copy Q's result to R1
    instr_origin.append('Termination_Q')
    idx += 1

    # Step 4: Adjust jump instructions
    adjusted_instructions = []
    for i, instr in enumerate(interleaved_instructions):
        if instr[0] == 3:  # Jumps
            m, n, k = instr[1:]
            origin = instr_origin[i]
            if origin == 'P':
                if k == 0:
                    new_k = t_p  # Jump to P's termination code
                else:
                    new_k = line_map_P.get(k, idx)  # Default to program end if not found
            elif origin == 'Q':
                if k == 0:
                    new_k = t_q  # Jump to Q's termination code
                else:
                    new_k = line_map_Q.get(k, idx)
            else:
                new_k = k  # For termination code, k remains the same
            adjusted_instr = ins.J(m, n, new_k)
            adjusted_instructions.append(adjusted_instr)
        else:
            adjusted_instructions.append(instr)

    return program(tuple(adjusted_instructions))


In [325]:
text_add = """
    C(2, 0)
    Z (2)

    J(1, 2, 0)  check loop condition
        S(0)
        S(2)
    J(0, 0, 3)  repeat loop
"""
add = program(compile(text_add))
run(add, 3, 4)

[0, 3, 4]
[4, 3, 4]
[4, 3, 0]
[4, 3, 0]
[5, 3, 0]
[5, 3, 1]
[5, 3, 1]
[5, 3, 1]
[6, 3, 1]
[6, 3, 2]
[6, 3, 2]
[6, 3, 2]
[7, 3, 2]
[7, 3, 3]
[7, 3, 3]
[7, 3, 3]


7

In [326]:
text_pred = """
    J(0, 1, 0)
    S(2)
    J(1, 2, 0)
        S(0)
        S(2)
    J(0, 0, 3)
"""
pred = program(compile(text_pred))
x = nat(5)
print(f"predecessor({x}) = {run(pred, x)}")

[0, 5]
[0, 5]
[0, 5, 1]
[0, 5, 1]
[1, 5, 1]
[1, 5, 2]
[1, 5, 2]
[1, 5, 2]
[2, 5, 2]
[2, 5, 3]
[2, 5, 3]
[2, 5, 3]
[3, 5, 3]
[3, 5, 4]
[3, 5, 4]
[3, 5, 4]
[4, 5, 4]
[4, 5, 5]
[4, 5, 5]
[4, 5, 5]
predecessor(5) = 4


In [327]:
a = alter(pred, add, )
print(a)
run(a, 4, 0, 0, 2, 9, )

  1: J(0, 1, 13)
  2: C(5, 3)
  3: S(2)
  4: Z(5)
  5: J(1, 2, 13)
  6: J(4, 5, 14)
  7: S(0)
  8: S(3)
  9: S(2)
 10: S(5)
 11: J(0, 0, 5)
 12: J(3, 3, 6)
 13: C(0, 0)
 14: C(3, 1)
[0, 4, 0, 0, 2, 9]
[0, 4, 0, 0, 2, 9]
[0, 4, 0, 9, 2, 9]
[0, 4, 1, 9, 2, 9]
[0, 4, 1, 9, 2, 0]
[0, 4, 1, 9, 2, 0]
[0, 4, 1, 9, 2, 0]
[1, 4, 1, 9, 2, 0]
[1, 4, 1, 10, 2, 0]
[1, 4, 2, 10, 2, 0]
[1, 4, 2, 10, 2, 1]
[1, 4, 2, 10, 2, 1]
[1, 4, 2, 10, 2, 1]
[1, 4, 2, 10, 2, 1]
[2, 4, 2, 10, 2, 1]
[2, 4, 2, 11, 2, 1]
[2, 4, 3, 11, 2, 1]
[2, 4, 3, 11, 2, 2]
[2, 4, 3, 11, 2, 2]
[2, 4, 3, 11, 2, 2]
[2, 4, 3, 11, 2, 2]
[2, 11, 3, 11, 2, 2]


2

In [317]:
print(a)

  1: J(0, 1, 13)
  2: C(5, 3)
  3: S(2)
  4: Z(5)
  5: J(1, 2, 13)
  6: J(4, 5, 14)
  7: S(0)
  8: S(3)
  9: S(2)
 10: S(5)
 11: J(0, 0, 5)
 12: J(3, 3, 6)
 13: C(0, 0)
 14: C(3, 1)


In [277]:
# 运行前驱程序和加法程序的交替执行
# 假设前驱程序 `pred` 接受一个输入 x，并返回 x-1
# 加法程序 `add` 接受两个输入 y 和 z，并返回 y + z
# 例如，运行 pred(5) 和 add(2, 3)

# 设置输入寄存器
m = memory()
m.write(1, 5)  # pred 输入
m.write(2, 2)  # add 第一个输入
m.write(3, 3)  # add 第二个输入

# 运行合并后的程序
ic = 1
while True:
    instr = altered_program.get_instruction(ic)
    if instr is None:
        break
    q, m = apply(instr, to=m)
    ic = ic + 1 if q is None else q

# 读取结果
result_p = m.read(0)  # R0 应为 pred(5) = 4
result_q = m.read(1)  # R1 应为 add(2,3) = 5

print(f"predecessor(5) = {result_p}")
print(f"add(2, 3) = {result_q}")


KeyboardInterrupt: 

In [309]:
def alter(P: program, Q: program) -> program:
    """
    The function returns the program implementing the following algorithm:
        1. Executes P and Q in disjoint memory segments.
        2. Alternately executes one instruction of P and one of Q.
        3. Checks after each instruction if P or Q has terminated.
        4. If either has terminated, returns the corresponding result.
    Arguments:
        P: program
        Q: program
    Returns:
        program
    """
    if not (isinstance(P, program) and isinstance(Q, program)):
        raise ValueError("alter() error! Invalid program type")

    # Determine the highest register used by P
    max_reg_P = P.haddr
    # Move Q's registers to avoid overlap with P
    offset_Q = max_reg_P + 5  # Extra space for instruction counters and flags

    # Move P and Q to their respective memory segments
    P_moved = move(P, off=5)  # Offset P by 5 to reserve space for counters
    Q_moved = move(Q, off=offset_Q)

    # Adjust instruction numbers in P and Q
    P_len = P.length
    Q_len = Q.length

    # Prepare instruction counters and termination flags
    # R1: ic_p (Instruction Counter for P)
    # R2: ic_q (Instruction Counter for Q)
    # R3: term_p (Termination flag for P)
    # R4: term_q (Termination flag for Q)
    # R0: Output register

    # Initialize instruction counters to 1
    init_ic = [ins.Z(1), ins.S(1), ins.Z(2), ins.S(2)]
    # Initialize termination flags to 0
    init_flags = [ins.Z(3), ins.Z(4)]

    # Main loop start label
    loop_start = len(init_ic) + len(init_flags) + 1

    # Build the main loop
    main_loop = []

    # Step 1: Execute one instruction of P
    main_loop.extend(execute_step(P_moved, ic_reg=1, term_flag=3, offset=5))

    # Check if P has terminated
    main_loop.extend(check_termination(P_len, ic_reg=1, term_flag=3, result_offset=5))

    # If P has terminated, output result and halt
    main_loop.extend(output_and_halt(term_flag=3, result_reg=5))

    # Step 2: Execute one instruction of Q
    main_loop.extend(execute_step(Q_moved, ic_reg=2, term_flag=4, offset=offset_Q))

    # Check if Q has terminated
    main_loop.extend(check_termination(Q_len, ic_reg=2, term_flag=4, result_offset=offset_Q))

    # If Q has terminated, output result and halt
    main_loop.extend(output_and_halt(term_flag=4, result_reg=offset_Q))

    # Jump back to the start of the loop
    loop_end = len(init_ic) + len(init_flags) + len(main_loop) + 1
    main_loop.append(ins.J(0, 0, loop_start))

    # Combine all parts to form the final program
    final_program = program(tuple(init_ic + init_flags + main_loop))

    return final_program

def execute_step(prog: program, ic_reg: int, term_flag: int, offset: int) -> List[ins]:
    """
    Simulate the execution of one instruction from the program.
    Arguments:
        prog: The moved program to execute.
        ic_reg: The register holding the instruction counter.
        term_flag: The termination flag register.
        offset: Memory offset for the program.
    Returns:
        List of instructions to execute one step.
    """
    instructions = []
    # Fetch the current instruction based on ic_reg
    # For simplicity, we assume instructions are stored in the program code
    # Since URM cannot index into instructions directly, we need to simulate this
    # This is a complex task in URM and may require encoding instructions into registers
    # For the sake of this example, we will assume this is handled
    # Note: In practice, simulating this in URM is non-trivial and may not be feasible
    return instructions

def check_termination(prog_len: int, ic_reg: int, term_flag: int, result_offset: int) -> List[ins]:
    """
    Check if the program has terminated by comparing the instruction counter.
    Arguments:
        prog_len: Length of the program.
        ic_reg: The instruction counter register.
        term_flag: The termination flag register.
        result_offset: Offset where the result is stored.
    Returns:
        List of instructions to check termination.
    """
    instructions = []
    # Compare ic_reg with prog_len + 1
    # If ic_reg == prog_len + 1, set term_flag to 1
    # Else, do nothing
    # Since URM doesn't support direct constants, we need to build prog_len + 1 into a register
    # Let's assume R_temp holds prog_len + 1
    R_temp = term_flag + 1  # Temporary register for comparison

    # Initialize R_temp with prog_len + 1
    instructions.extend([ins.Z(R_temp)])
    for _ in range(prog_len + 1):
        instructions.append(ins.S(R_temp))

    # Compare ic_reg and R_temp
    instructions.append(ins.J(ic_reg, R_temp, len(instructions) + 3))
    # If not equal, jump over the next instruction
    instructions.append(ins.Z(R_temp))  # Not equal, do nothing
    # If equal, set term_flag to 1
    instructions.append(ins.S(term_flag))

    return instructions

def output_and_halt(term_flag: int, result_reg: int) -> List[ins]:
    """
    If the termination flag is set, copy the result to R0 and halt.
    Arguments:
        term_flag: The termination flag register.
        result_reg: The register containing the result.
    Returns:
        List of instructions to output the result and halt.
    """
    instructions = []
    # If term_flag == 1, copy result to R0 and halt
    current_line = len(instructions) + 1
    # If term_flag != 1, jump over the next instructions
    instructions.append(ins.J(term_flag, term_flag, current_line + 3))
    # Copy result to R0
    instructions.append(ins.C(result_reg, 0))
    # Halt by jumping to itself
    instructions.append(ins.J(0, 0, len(instructions)))
    return instructions


In [308]:
# Move pred to start from register 10
offset_pred = 10
pred_moved = move(pred, off=offset_pred)
print("Adjusted 'add' Program:")
print(add)
print("\nAdjusted 'pred' Program (moved to offset 10):")
print(pred_moved)


Adjusted 'add' Program:
  1: C(2, 0)
  2: Z(2)
  3: J(1, 2, 0)
  4: S(0)
  5: S(2)
  6: J(0, 0, 3)

Adjusted 'pred' Program (moved to offset 10):
  1: J(10, 11, 0)
  2: S(12)
  3: J(11, 12, 0)
  4: S(10)
  5: S(12)
  6: J(10, 10, 3)
